In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# 输入研究区的shp文件所在路径
wuhan18_shp ="D:/NDVI_exercise/wuhan/wuhan1+8t.shp"
# shp 转 ee 
wuhan18_ee = geemap.shp_to_ee(wuhan18_shp)
# 添加到layer
Map.addLayer(wuhan18_ee,{},"wuhan1+8")
Map.centerObject(wuhan18_ee,7)   # 设置以研究区居中

In [4]:
def AWEI(image):
    awei = image.expression('(4 * (green - swir1)) - (0.25 * nir + 2.75 * swir2)',
        {
          'green': image.select('B3'),    
          'nir': image.select('B5'),    
          'swir1': image.select('B6'),
          'swir2': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

In [5]:
# 筛选数据（使用日期，云量（小于5%），研究区），去云
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan18_ee) \
    .filterDate('2019-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER',5)) \
#   .map(maskL8sr)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

In [6]:
landsat_map = landsat_images.map(AWEI)
wuhan18_landsat = landsat_map.mean().clip(wuhan18_ee)
Map.addLayer(wuhan18_landsat,vis_params,'wuhan18 landsat')

In [7]:
water_awei = wuhan18_landsat.select('AWEI').gt(0).selfMask()

Map.addLayer(water_awei,{'palette': 'blue'},'water_awei')